<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Contract_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WithPi Contract Calibration

This colab assumes that you already went through [Input Generation](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Input_Generation.ipynb), and now wish to calibrate your prompt.

We will walk through the same `Aesop AI` example, but you can load any contract here. Let's dig in!

This should take about **15 minutes**, even if you're unfamiliar with Colab.

## Install and initialize SDK

Connect to a regular CPU Python 3 runtime.  You won't need GPUs for this notebook.

You'll need a WITHPI_API_KEY from https://play.withpi.ai.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [2]:
%%capture

%pip install withpi httpx pandas

import os
from google.colab import userdata
from withpi import PiClient

os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

client = PiClient()

# Load contract and Dataset

Load the `Aesop AI` example and example set from Pi Labs cookbooks, or edit below to load a different one.


In [3]:
import httpx
import pandas as pd
from google.colab import data_table
from withpi.types import Contract

resp = httpx.get("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/contracts/aesop_ai.json")

aesop_contract = Contract.model_validate_json(resp.content)

for dimension in aesop_contract.dimensions:
  print(dimension.label)
  for sub_dimension in dimension.sub_dimensions:
    print(f"\t{sub_dimension.description}")

df = pd.read_parquet("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/datasets/aesop_ai_examples.parquet")
data_table.enable_dataframe_formatter()
df


Story Structure
	Does the story have a clear beginning, middle, and end?
	Is there a conflict introduced early in the story that drives the plot?
	Is the resolution of the conflict clear and satisfying?
Character Development
	Does the story include characters that are relatable for children?
	Do the characters demonstrate growth or change by the end of the story?
	Is the dialogue between characters natural and age-appropriate?
Narrative Engagement
	Is the story engaging and likely to hold a child's interest?
	Does the story use vivid imagery to help children visualize the scenes?
	Does the story incorporate repetitive elements that aid in comprehension and retention?
Language Appropriateness
	Is the language used in the story appropriate for children's understanding?
	Is the story an appropriate length for a children's tale (not exceeding 1000 words)?
Moral and Cultural Sensitivity
	Does the story clearly convey a specified life lesson?
	Does the story reiterate the moral lesson at the

,input,output
0,Write a children's story in the style of Aesop...,Barnaby the hare was a blur of twitching whisk...
1,Tell a fable about a crow and a fox that illus...,"Once upon a time, in a sun-drenched forest, li..."
2,Create a story featuring a lion and a mouse th...,"Leo the lion, king of the sprawling savanna, w..."
3,Write a fable involving a tortoise and a hare ...,The Tortoise and the Determined Hare\n\nIn the...
4,Tell a story about a greedy dog who loses his ...,Barnaby the Beagle was a dog of magnificent ap...
5,Spin a tale with a squirrel and an owl teachin...,Barnaby the squirrel was renowned throughout t...
6,Compose a fable with a feuding sun and wind th...,"The Sun and the Wind\n\nThe Sun, a fiery ball ..."
7,Dream up a story involving a hummingbird and a...,"Pip the hummingbird, a flash of emerald and ru..."
8,Tell a saga with a rabbit and a cunning crow i...,Barnaby the rabbit was a champion hopper. He'...
9,Craft a fable about a young rabbit needing hel...,Barnaby Bunson was a young rabbit with a very ...


## Cluster Inputs

We're going to label some inputs as "good" and "bad", but to do this it is helpful to focus on a few different types of input.  We'll use clustering to make sure we don't have to look at too many examples.

In [5]:
input_topic_clusters = client.data.inputs.cluster(
    inputs=[{"identifier": str(index), "llm_input": row["input"]} for index, row in df.iterrows()],
)

df['cluster'] = ['']*len(df)
for cluster in input_topic_clusters:
  for identifier in cluster.inputs:
    df.loc[int(identifier),'cluster'] = cluster.topic
df

,input,output,cluster
0,Write a children's story in the style of Aesop...,Barnaby the hare was a blur of twitching whisk...,Fables Featuring Tortoises and Lessons
1,Tell a fable about a crow and a fox that illus...,"Once upon a time, in a sun-drenched forest, li...",Animal Fables Teaching Life Lessons
2,Create a story featuring a lion and a mouse th...,"Leo the lion, king of the sprawling savanna, w...",Repeated Theme-Based Story Prompts
3,Write a fable involving a tortoise and a hare ...,The Tortoise and the Determined Hare\n\nIn the...,Fables Featuring Tortoises and Lessons
4,Tell a story about a greedy dog who loses his ...,Barnaby the Beagle was a dog of magnificent ap...,Animal Fables Teaching Life Lessons
5,Spin a tale with a squirrel and an owl teachin...,Barnaby the squirrel was renowned throughout t...,Animal Fables Teaching Life Lessons
6,Compose a fable with a feuding sun and wind th...,"The Sun and the Wind\n\nThe Sun, a fiery ball ...",Repeated Theme-Based Story Prompts
7,Dream up a story involving a hummingbird and a...,"Pip the hummingbird, a flash of emerald and ru...",Animal Fables Teaching Life Lessons
8,Tell a saga with a rabbit and a cunning crow i...,Barnaby the rabbit was a champion hopper. He'...,Animal Fables Teaching Life Lessons
9,Craft a fable about a young rabbit needing hel...,Barnaby Bunson was a young rabbit with a very ...,Animal Fables Teaching Life Lessons


## Identify outliers

Let's first score every input against the contract, adding that as a column.  Pi scoring is fast enough that serially processing the dataset is fine.

In [7]:
df["uncalibrated_scores"] = [client.contracts.score(contract=aesop_contract, llm_input=row["input"], llm_output=row["output"]).total_score for idx, row in df.iterrows()]
df

,input,output,cluster,uncalibrated_scores
0,Write a children's story in the style of Aesop...,Barnaby the hare was a blur of twitching whisk...,Fables Featuring Tortoises and Lessons,0.457066
1,Tell a fable about a crow and a fox that illus...,"Once upon a time, in a sun-drenched forest, li...",Animal Fables Teaching Life Lessons,0.437156
2,Create a story featuring a lion and a mouse th...,"Leo the lion, king of the sprawling savanna, w...",Repeated Theme-Based Story Prompts,0.425824
3,Write a fable involving a tortoise and a hare ...,The Tortoise and the Determined Hare\n\nIn the...,Fables Featuring Tortoises and Lessons,0.432215
4,Tell a story about a greedy dog who loses his ...,Barnaby the Beagle was a dog of magnificent ap...,Animal Fables Teaching Life Lessons,0.456638
5,Spin a tale with a squirrel and an owl teachin...,Barnaby the squirrel was renowned throughout t...,Animal Fables Teaching Life Lessons,0.444735
6,Compose a fable with a feuding sun and wind th...,"The Sun and the Wind\n\nThe Sun, a fiery ball ...",Repeated Theme-Based Story Prompts,0.471554
7,Dream up a story involving a hummingbird and a...,"Pip the hummingbird, a flash of emerald and ru...",Animal Fables Teaching Life Lessons,0.423276
8,Tell a saga with a rabbit and a cunning crow i...,Barnaby the rabbit was a champion hopper. He'...,Animal Fables Teaching Life Lessons,0.480921
9,Craft a fable about a young rabbit needing hel...,Barnaby Bunson was a young rabbit with a very ...,Animal Fables Teaching Life Lessons,0.481269


## Label data

Now it's time to label examples against a simple statement.  **The response fully satisfies the input according to the contract**.  Valid responses are **Strongly Agree**, **Agree**, **Neutral**, **Disagree**, and **Strongly Disagree**, or simply **5** down to **1**.

The below cell will select a high and low scoring exemplar from each cluster, asking you to respond **5** through **1**

In [ ]:
from pathlib import Path
from google.colab import files

filename = 'aesop_ai_improved_prompt.json'
Path(filename).write_text(aesop_contract.model_dump_json(indent=2))
files.download(filename)

## Next Steps

Now that you have an improved prompt using an **uncalibrated** contract, let's try **calibrating** the contract!  This should help it more closely align with what you actually value.  Proceed on to the [Contract Calibration](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Contract_Calibration.ipynb) colab to do this.